In [1]:
import mat73
data = mat73.loadmat('long_term_data/Chewie_CO_2016/Chewie_20160927_001.mat')
print(data.keys())
print(data['xds'].keys())

dict_keys(['xds'])
dict_keys(['bin_width', 'curs_a', 'curs_p', 'curs_v', 'has_EMG', 'has_cursor', 'has_force', 'has_raw_EMG', 'has_raw_force', 'meta', 'sorted', 'spike_counts', 'spikes', 'time_frame', 'trial_end_time', 'trial_gocue_time', 'trial_info_table', 'trial_info_table_header', 'trial_result', 'trial_start_time', 'trial_target_corners', 'trial_target_dir', 'unit_names'])


In [2]:
data['xds']['spike_counts'].shape

(869105, 96)

In [3]:
data['xds']['has_force']

False

In [4]:
import numpy as np

# 准备表格头
print(f"{'变量名 (Key)':<25} | {'类型/维度 (Shape/Value)':<30}")
print("-" * 60)

xds = data['xds']
for key in xds.keys():
    val = xds[key]
    
    # 判断是否为 numpy 数组或大型列表
    if isinstance(val, (np.ndarray, list)):
        # 如果是数组，打印它的 shape
        if hasattr(val, 'shape'):
            if val.shape == ():
                info = f"Scalar: {val}"
            else:
                info = f"Array Shape: {val.shape}"
        else:
            info = f"List Length: {len(val)}"
    else:
        # 如果是标量或小型元数据，直接打印值
        info = str(val)

    print(f"{key:<25} | {info:<30}")

变量名 (Key)                 | 类型/维度 (Shape/Value)           
------------------------------------------------------------
bin_width                 | Scalar: 0.001                 
curs_a                    | Array Shape: (869105, 2)      
curs_p                    | Array Shape: (869105, 2)      
curs_v                    | Array Shape: (869105, 2)      
has_EMG                   | False                         
has_cursor                | True                          
has_force                 | False                         
has_raw_EMG               | Scalar: 0.0                   
has_raw_force             | Scalar: 0.0                   
meta                      | {'aliasList': [], 'array': 'M1', 'cdsName': 'Chewie_M1_CO_2016-9-27_lab1', 'cdsVersion': array(0.), 'dataSource': 'NEVNSx', 'dataWindow': array([2.00000e-03, 8.69106e+02]), 'dateTime': '2016/9/27 21:1:18.825', 'duration': array(869.104), 'hasAnalog': True, 'hasBumps': True, 'hasChaoticLoad': False, 'hasEmg': False, 'has

In [6]:
import numpy as np

# 列出数组中所有出现过的值
unique_values = np.unique(data['xds']['spike_counts'])
print(unique_values)

[0. 1.]
